# Predicting whether or not a student will be placed (Logistic Regression)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
dataset = pd.read_excel('placement.xlsx')
dataset.head(10)

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,CGPA,degree_t,workex,etest_p,specialisation,Masters,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.00,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.50,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.00,Mkt&Fin,57.80,Placed,250000.0
3,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.80,Mkt&Fin,55.50,Placed,425000.0
4,8,M,82.00,Central,64.00,Central,Science,66.00,Sci&Tech,Yes,67.00,Mkt&Fin,62.14,Placed,252000.0
5,9,M,73.00,Central,79.00,Central,Commerce,72.00,Comm&Mgmt,No,91.34,Mkt&Fin,61.29,Placed,231000.0
6,11,M,58.00,Central,61.00,Central,Commerce,60.00,Comm&Mgmt,Yes,62.00,Mkt&HR,60.85,Placed,260000.0
7,12,M,69.60,Central,68.40,Central,Commerce,78.30,Comm&Mgmt,Yes,60.00,Mkt&Fin,63.70,Placed,250000.0
8,14,F,77.00,Central,87.00,Central,Commerce,59.00,Comm&Mgmt,No,68.00,Mkt&Fin,68.63,Placed,218000.0
9,16,F,65.00,Central,75.00,Central,Commerce,69.00,Comm&Mgmt,Yes,72.00,Mkt&Fin,64.66,Placed,200000.0


In [5]:
dataset = dataset.drop('sl_no', axis=1)
dataset = dataset.drop('salary', axis=1)

In [14]:
dataset["gender"] = dataset["gender"].astype('category')
dataset["ssc_b"] = dataset["ssc_b"].astype('category')
dataset["hsc_b"] = dataset["hsc_b"].astype('category')
dataset["degree_t"] = dataset["degree_t"].astype('category')
dataset["workex"] = dataset["workex"].astype('category')
dataset["specialisation"] = dataset["specialisation"].astype('category')
dataset["status"] = dataset["status"].astype('category')
dataset["hsc_s"] = dataset["hsc_s"].astype('category')
dataset.dtypes

gender            category
ssc_p              float64
ssc_b             category
hsc_p              float64
hsc_b             category
hsc_s             category
CGPA               float64
degree_t          category
workex            category
etest_p            float64
specialisation    category
Masters            float64
status            category
dtype: object

In [15]:
dataset["gender"] = dataset["gender"].cat.codes
dataset["ssc_b"] = dataset["ssc_b"].cat.codes
dataset["hsc_b"] = dataset["hsc_b"].cat.codes
dataset["degree_t"] = dataset["degree_t"].cat.codes
dataset["workex"] = dataset["workex"].cat.codes
dataset["specialisation"] = dataset["specialisation"].cat.codes
dataset["status"] = dataset["status"].cat.codes
dataset["hsc_s"] = dataset["hsc_s"].cat.codes

dataset.head(20)

,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,CGPA,degree_t,workex,etest_p,specialisation,Masters,status
0,1,67.00,1,91.00,1,1,58.00,2,0,55.00,1,58.80,1
1,1,79.33,0,78.33,1,2,77.48,2,1,86.50,0,66.28,1
2,1,65.00,0,68.00,0,0,64.00,0,0,75.00,0,57.80,1
3,1,85.80,0,73.60,0,1,73.30,0,0,96.80,0,55.50,1
4,1,82.00,0,64.00,0,2,66.00,2,1,67.00,0,62.14,1
5,1,73.00,0,79.00,0,1,72.00,0,0,91.34,0,61.29,1
6,1,58.00,0,61.00,0,1,60.00,0,1,62.00,1,60.85,1
7,1,69.60,0,68.40,0,1,78.30,0,1,60.00,0,63.70,1
8,0,77.00,0,87.00,0,1,59.00,0,0,68.00,0,68.63,1
9,0,65.00,0,75.00,0,1,69.00,0,1,72.00,0,64.66,1


In [16]:
X = dataset.iloc[:, :-1]
Y = dataset.iloc[:, -1]

Y

0      1
1      1
2      1
3      1
4      1
      ..
210    0
211    0
212    0
213    0
214    0
Name: status, Length: 215, dtype: int8

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

dataset.head()

,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,CGPA,degree_t,workex,etest_p,specialisation,Masters,status
0,1,67.00,1,91.00,1,1,58.00,2,0,55.0,1,58.80,1
1,1,79.33,0,78.33,1,2,77.48,2,1,86.5,0,66.28,1
2,1,65.00,0,68.00,0,0,64.00,0,0,75.0,0,57.80,1
3,1,85.80,0,73.60,0,1,73.30,0,0,96.8,0,55.50,1
4,1,82.00,0,64.00,0,2,66.00,2,1,67.0,0,62.14,1


In [22]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, solver='lbfgs', max_iter=1000).fit(X_train, Y_train)

clf.score(X_test, Y_test)

0.8837209302325582

In [23]:
Y_pred = clf.predict(X_test)
Y_pred

array([0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0],
      dtype=int8)

In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score

print(confusion_matrix(Y_test, Y_pred))

print(accuracy_score(Y_test, Y_pred))

[[10  1]
 [ 4 28]]
0.8837209302325582
